### 実験条件の設定

In [ ]:
import sys
# データ量を全体の1%, 20%, 60% 100%から選択する
UTIL_RATE=20
allowed_util_rate = [1, 20, 60, 100]

if(UTIL_RATE not in allowed_util_rate):
    print('データ量の設定が誤っています')
    sys.exit()

%env UTIL_RATE = $UTIL_RATE

### 割り当てられたGPUの確認

In [ ]:
!nvidia-smi

### 必要なライブラリのインストール

In [ ]:
%%bash
python3 -m pip install --user ConfigArgParse
python3 -m pip install --user torchtext==0.4.0

### パスなど環境変数の定義

In [ ]:
import os
pwd = os.getcwd()
### This directory contains libraries and exexution files.
APPS_DIR = pwd + '/apps'
%env APPS_DIR=$APPS_DIR
### This directory contains training, development and test set
DATA_DIR = pwd + '/dataset'
%env DATA_DIR=$DATA_DIR
### This directory contains output directions.
WORK_DIR = pwd + '/work'
%env WORK_DIR=$WORK_DIR

#japanese -> english
%env LANG_PAIR=ja-en
#english -> japanese
#%env LANG_PAIR=en-ja

%env DATASET=$DATA_DIR/kftt-data-1.0/data/tok


### 使用するライブラリのインストール

In [ ]:
%%bash
module load gcc

echo "download and make apps..."

mkdir -p ${APPS_DIR}
cd ${APPS_DIR}

git clone https://github.com/moses-smt/mosesdecoder.git
git clone https://github.com/OpenNMT/OpenNMT-py.git -b 1.2.0
git clone https://github.com/neubig/kytea.git
cd kytea
autoreconf -i
./configure --prefix=${APPS_DIR}/kytea
make
make install

echo "finish apps preparation."

### 使用するデータのダウンロードとデータ量調整

In [ ]:
%%bash

mkdir -p $WORK_DIR/input
mkdir -p ${DATA_DIR}

# Download data set
wget -O ${DATA_DIR}/kftt-data-1.0.tar.gz http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
cd ${DATA_DIR}
# Uncompress
tar xvzf kftt-data-1.0.tar.gz

paste \
  ${DATASET}/kyoto-train.cln.en \
  ${DATASET}/kyoto-train.cln.ja |
  shuf > ${DATASET}/train.shuf

if [ $UTIL_RATE == 1 ]; then
    echo "data size : 1%"
    head -n $((`cat ${DATASET}/train.shuf | wc -l`/100)) ${DATASET}/train.shuf | cut -f 1 > ${DATASET}/train.en
    head -n $((`cat ${DATASET}/train.shuf | wc -l`/100)) ${DATASET}/train.shuf | cut -f 2 > ${DATASET}/train.ja
elif [ $UTIL_RATE == 20 ]; then
    echo "data size : 20%"
    head -n $((`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 1 > ${DATASET}/train.en
    head -n $((`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 2 > ${DATASET}/train.ja
elif [ $UTIL_RATE == 60 ]; then
    echo "data size : 60%"
    head -n $((3*`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 1 > ${DATASET}/train.en
    head -n $((3*`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 2 > ${DATASET}/train.ja
else
    echo "data size : 100%"
    cut -f 1 ${DATASET}/train.shuf > ${DATASET}/train.en
    cut -f 2 ${DATASET}/train.shuf > ${DATASET}/train.ja
fi

### 使用するデータのサンプル

In [ ]:
%%bash

train_name=train
src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

sed -n 1P ${DATASET}/${train_name}.${src}
sed -n 1P ${DATASET}/${train_name}.${trg}
sed -n 2P ${DATASET}/${train_name}.${src}
sed -n 2P ${DATASET}/${train_name}.${trg}

### OpenNMT-pyライブラリで前処理

In [ ]:
%%bash

cd ${APPS_DIR}/OpenNMT-py
suffix="en-ja ja-en"
train_name=train
src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`
    python preprocess.py \
    -train_src ${DATASET}/${train_name}.${src} \
    -train_tgt ${DATASET}/${train_name}.${trg} \
    -valid_src ${DATASET}/kyoto-dev.${src} \
    -valid_tgt ${DATASET}/kyoto-dev.${trg} \
    -save_data ${DATA_DIR}/dicts-${train_name}-${LANG_PAIR} \
    -src_words_min_frequency 5 \
    -tgt_words_min_frequency 5 \
    -src_seq_length 40 \
    -tgt_seq_length 40
#train_src, train_tgt, valid_src, valid_tgt: text
#save_data: bin

### 前処理後の学習データ

In [ ]:
%cd {APPS_DIR}/OpenNMT-py

import torch
import os

data_path = os.path.join(DATA_DIR, 'dicts-train-ja-en.train.0.pt')
vocab_path = os.path.join(DATA_DIR, 'dicts-train-ja-en.vocab.pt')
#学習データ。
#data = torch.load('/content/dataset/dicts-train-ja-en.train.0.pt')
data = torch.load(data_path)

#vocab: データの種類とその処理方法の定義。
#vocab = torch.load('/content/dataset/dicts-train-ja-en.vocab.pt')
vocab = torch.load(vocab_path)

print("学習データ数", len(data))
print("")
print("data[0]")
print("index", data[0].indices)
print("翻訳前", data[0].src)
print("翻訳後", data[0].tgt)
print("")
print("data[1]")
print("index", data[1].indices)
print("翻訳前", data[1].src)
print("翻訳後", data[1].tgt)
print("")
print("これが数値に変換され、以下の形式でニューラルネットに渡される。")
print(vocab['src'].fields[0][1].process((data[0].src[0],data[1].src[0])))

### ニューラルネットの学習

In [ ]:
%%bash

start_time=`date +%s`

TIME_PATH=$(pwd)/train.time.log



TRAIN_NAME=train
MODEL_DIR=${WORK_DIR}/models

NUM_EPOCH=10

mkdir -p ${MODEL_DIR}

NUM_DATA=$(cat ${DATASET}/${TRAIN_NAME}.en | wc -l)

cd ${APPS_DIR}/OpenNMT-py

batch_size=256
num_steps=$((${NUM_DATA}*${NUM_EPOCH}/${batch_size}))
#モデルは左のファイルボタンから、
#encoder: apps/OpenNMT-py/onmt/encoders/rnn_encoder.py (RNNEncoder)
#decoder: apps/OpenNMT-py/onmt/encoders/decoder.py (StdRNNDecoder)
#を参照。または授業資料を参照。
python train.py \
    -data ${DATA_DIR}/dicts-${TRAIN_NAME}-${LANG_PAIR} \
    -save_model ${MODEL_DIR}/${LANG_PAIR} \
    -layers 2 \
    -rnn_size 500 \
    -word_vec_size 300 \
    -optim adam \
    -learning_rate 0.001 \
    -dropout 0.3 \
    -batch_size ${batch_size} \
    -report_every 1 \
    -save_checkpoint_steps ${num_steps} \
    -train_steps ${num_steps} \
    -gpu_rank 0 |& awk '(NR%30==0 || NR < 70){print}' #出力を少し抑制（重くなるので）
cp ${MODEL_DIR}/${LANG_PAIR}_step_${num_steps}.pt ${MODEL_DIR}/${LANG_PAIR}_final.pt


end_time=`date +%s`
time=$((end_time - start_time))
echo "${time} (sec)" >& $TIME_PATH

### 結果の評価

In [ ]:
%%bash


MODELS=${WORK_DIR}/models
OUT_DIR=${WORK_DIR}/outputs
USRDIR=${WORK_DIR}/input
SCORE_PATH=$(pwd)/score.txt

mkdir -p ${OUT_DIR}

cd ${APPS_DIR}/OpenNMT-py

src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

#学習したモデルを使ってsize1のビームサーチにより翻訳結果を取得。
python translate.py \
    -model ${MODELS}/${LANG_PAIR}_final.pt \
    -src ${DATASET}/kyoto-test.${src} \
    -output ${OUT_DIR}/test.${trg} \
    -gpu 0 \
    -beam_size 1 \
    -batch_size 512 \
    -verbose

### 好きな文章を翻訳してみる
#### （以下の`echo 何らかの文章をここに書く。 > ${USRDIR}/user.${src}`を書き換えて上のセル実行ボタン▶を押してみよう）

In [ ]:
%%bash


MODELS=${WORK_DIR}/models
OUT_DIR=${WORK_DIR}/outputs
USRDIR=${WORK_DIR}/input
SCORE_PATH=$(pwd)/score.txt

cd ${APPS_DIR}/OpenNMT-py

src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

for i in 1 2 3 4 5; do
  echo [評価データの翻訳結果 $i]
  sed -n ${i}P ${OUT_DIR}/test.${trg}
  echo [評価データの正しい翻訳結果 $i]
  sed -n ${i}P ${DATASET}/kyoto-test.${trg}
done

#bleuスコア算出。
perl ${APPS_DIR}/mosesdecoder/scripts/generic/multi-bleu.perl \
    ${DATASET}/kyoto-test.${trg} \
    < ${OUT_DIR}/test.${trg} \
    1> ${OUT_DIR}/result_${LANG_PAIR}.bleu \
    2> /dev/null

cat ${OUT_DIR}/result_${LANG_PAIR}.bleu | sed -r 's/(BLEU = [0-9]*\.[0-9]*), .*/\1/g' | tee ${SCORE_PATH}

In [ ]:
%%bash

MODELS=${WORK_DIR}/models
OUT_DIR=${WORK_DIR}/outputs
USRDIR=${WORK_DIR}/input
SCORE_PATH=$(pwd)/score.txt
src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

echo 何らかの文章をここに書く。 > ${USRDIR}/user.${src}

if [ ! -e ${USRDIR}/user.${src} ]
then
    echo "${USRDIR}/user.${src} does not exist."
    exit
fi


#tokenに分割。
if [ ${src} = "ja" -a ${src} != "en" ]
then
    # Tokenize Japanese sentences
    bash ${APPS_DIR}/kytea/src/bin/kytea \
    < ${USRDIR}/user.${src} |\
    sed 's/\/[^ ]\+//g' \
    > ${USRDIR}/user.tok.${src}
elif [ ${src} = "en" -a ${src} != "ja" ]
then
    # Tokenize English sentences
    perl ${APPS_DIR}/mosesdecoder/scripts/tokenizer/tokenizer.perl \
    -l en \
    < ${USRDIR}/user.${src} \
    > ${USRDIR}/user.tok.${src}
else
echo "Language: ${src} is undefined."
    continue
fi

#分割結果。
cat ${USRDIR}/user.tok.${src}

#あとは同様に翻訳する。
python translate.py \
    -model ${MODELS}/${LANG_PAIR}_final.pt \
    -src ${USRDIR}/user.tok.${src} \
    -output ${OUT_DIR}/user.${trg} \
    -gpu 0 \
    -beam_size 1 \
    -batch_size 512 \
    -verbose

In [ ]:
!cat train.time.log
!cat score.txt